In [46]:
import dadi
import ipywidgets as widgets
from ipywidgets import FloatSlider, interact_manual, interact

from IPython.display import display
from Models import get_dadi_model_func, get_dadi_model_params

Upload data

In [2]:
upload = widgets.FileUpload(
               accept='',  # Accepted file extension e.g. '.fs'
               multiple=False  # True to accept multiple files upload else False
           )
display(upload)

FileUpload(value={}, description='Upload')

In [3]:
# Load the data from user input
uploaded_filename = next(iter(upload.value))
content = upload.value[uploaded_filename]['content']
with open(uploaded_filename, 'wb') as f: f.write(content)
# Read data into dadi
data = dadi.Spectrum.from_file(uploaded_filename)
ns = data.sample_sizes

Select grid size

In [4]:
a = widgets.BoundedIntText(
    value=40,
    min=1,
    max=10000,
    step=1,
    description='Grid-x:',
    disabled=False
)
b = widgets.BoundedIntText(
    value=50,
    min=1,
    max=10000,
    step=1,
    description='Grid-y:',
    disabled=False
)
c = widgets.BoundedIntText(
    value=60,
    min=1,
    max=10000,
    step=1,
    description='Grid-z:',
    disabled=False
)

ui = widgets.HBox([a, b, c])

def f(a, b, c):
    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()

In [5]:
# These are the grid point settings will use for extrapolation.
pts_l = [a.value, b.value, c.value]

Select demographic models

In [6]:
model = widgets.Dropdown(
    options=['bottlegrowth_1d', 'growth_1d', 'snm_1d', 'three_epoch_1d', 'two_epoch_1d',
             'bottlegrowth_2d', 'bottlegrowth_split', 'bottlegrowth_split_mig', 'IM', 'IM_pre', 'snm_2d', 'split_mig', 'split_asym_mig'],
    description='Model:',
)
display(model)

Dropdown(description='Model:', options=('bottlegrowth_1d', 'growth_1d', 'snm_1d', 'three_epoch_1d', 'two_epoch…

In [10]:
func = get_dadi_model_func(model.value)

In [11]:
# Make the extrapolating version of our demographic model function.
func_ex = dadi.Numerics.make_extrap_log_func(func)

Select upper bounds

In [12]:
params = get_dadi_model_params(model.value)
sliders = []
for p in params:
    if p == 's':
        s = widgets.BoundedFloatText(
            value=0.5,
            min=0,
            max=1,
            step=0.1,
            description=p+':',
        )
    else:
        s = widgets.BoundedFloatText(
                value=10,
                min=0,
                max=10000,
                step=0.1,
                description=p+':',
        )
    sliders.append(s)
print(sliders)    
ui = widgets.VBox(sliders)
display(ui)

[BoundedFloatText(value=10.0, description='nuB:', max=10000.0, step=0.1), BoundedFloatText(value=10.0, description='nuF:', max=10000.0, step=0.1), BoundedFloatText(value=10.0, description='m:', max=10000.0, step=0.1), BoundedFloatText(value=10.0, description='T:', max=10000.0, step=0.1), BoundedFloatText(value=10.0, description='Ts:', max=10000.0, step=0.1)]


In [13]:
upper_bound = []
for s in sliders:
    upper_bound.append(s.value)

Select lower bounds

In [14]:
params = get_dadi_model_params(model.value)
sliders = []
for p in params:
    if p == 's':
        s = widgets.BoundedFloatText(
            value=0.5,
            min=0,
            max=1,
            step=0.1,
            description=p+':',
        )
    else:
        s = widgets.BoundedFloatText(
                value=0,
                min=0,
                max=10000,
                step=0.1,
                description=p+':',
        )
    sliders.append(s)
    
ui = widgets.VBox(sliders)
display(ui)

In [15]:
lower_bound = []
for s in sliders:
    lower_bound.append(s.value)

Ineractive plots to choose initial p0

In [103]:
def plot_func_2d(**sliders_dict):    
    p0 = [x for x in sliders_dict.values()]
    print('Here are your model ('+model.value+') parameters:')
    print(p0)
    test_model = func_ex(p0, ns, pts_l)
    dadi.Plotting.plot_2d_comp_multinom(test_model, data, vmin=1, resid_range=3, show=True)

def plot_func_1d(**sliders_dict):
    p0 = [x for x in sliders_dict.values()]
    print('Here are your model ('+model.value+') parameters:')
    print(p0)
    test_model = func_ex(p0, ns, pts_l)
    dadi.Plotting.plot_1d_comp_multinom(test_model, data, vmin=1, resid_range=3, show=True)

    
sliders_dict = {}
for p, l, u in zip(params, lower_bound, upper_bound):
    sliders_dict[p+':'] = FloatSlider(
        value=0.5,
        min=l,
        max=u,
        step=0.1,
    )

if '1d' not in model.value:
    interact_manual(plot_func_2d, **sliders_dict)
else:
    interact_manual(plot_func_1d, **sliders_dict)
    

interactive(children=(FloatSlider(value=0.5, description='nuB:', max=10.0), FloatSlider(value=0.5, description…